In [1]:
# Importar librerías
from os.path import exists, isfile
import pandas as pd
import numpy as np

# Rutas a los archivos CSV a cargar
PATH_CSV_VENTAS = 'sales.csv'
PATH_CSV_INVENTARIO = 'inventories.csv'
PATH_CSV_SATISFACCION = 'satisfaction.csv'

def verificar_fichero(path_fichero: str) -> None:
    """Verifica si un fichero existe y es fichero (i.e., no es un directorio). Si no se cumple alguna de estas dos condiciones, se lanza una excepción SystemExit para interrumpir la ejecución del iPythonNotebook."""
    if not exists(path_fichero) or not isfile(path_fichero):
        raise SystemExit('Alguno de los CSVs de entrada no existe.')

# Verificar si los archivos existen
verificar_fichero(PATH_CSV_VENTAS)
verificar_fichero(PATH_CSV_INVENTARIO)
verificar_fichero(PATH_CSV_SATISFACCION)

# Cargar datos en DataFrames (eliminando filas con valores nulos)
ventas = pd.read_csv(PATH_CSV_VENTAS).dropna()
inventario = pd.read_csv(PATH_CSV_INVENTARIO).dropna()
satisfaccion = pd.read_csv(PATH_CSV_SATISFACCION).dropna()

In [2]:
# 4. Exploración de Datos #

# Ventas totales por tienda y por categoría de producto
print('Ventas totales por tienda:')
print(ventas.groupby(['ID_Tienda'])[['Cantidad_Vendida']].sum())
print('\nVentas totales por producto:')
print(ventas.groupby(['Producto'])[['Cantidad_Vendida']].sum())

# Ingresos totales por tienda
ventas['Ingresos'] = ventas['Cantidad_Vendida'] * ventas['Precio_Unitario']
print('\nIngresos totales por tienda:')
print(ventas.groupby('ID_Tienda')['Ingresos'].sum())

# Mostrar estadísticas de cada DataFrame
print('\nEstadísticas de las ventas:')
print(ventas.describe())
print('\nEstadísticas del inventario:')
print(inventario.describe())
print('\nEstadísticas de la satisfacción:')
print(satisfaccion.describe())

# Ventas medias por tienda y por categoría de producto
print('\nVentas medias por tienda:')
print(ventas.groupby(['ID_Tienda'])[['Cantidad_Vendida']].mean())
print('\nVentas medias por categoría de producto:')
print(ventas.groupby(['Producto'])[['Cantidad_Vendida']].mean())

Ventas totales por tienda:
           Cantidad_Vendida
ID_Tienda                  
1                        35
2                        55
3                        50
4                        60
5                        59

Ventas totales por producto:
            Cantidad_Vendida
Producto                    
Producto A                85
Producto B                75
Producto C                90
Producto D                 9

Ingresos totales por tienda:
ID_Tienda
1     5000
2    10500
3     9000
4    13000
5    15700
Name: Ingresos, dtype: int64

Estadísticas de las ventas:
       ID_Tienda  Cantidad_Vendida  Precio_Unitario      Ingresos
count  11.000000         11.000000        11.000000     11.000000
mean    3.181818         23.545455       200.000000   4836.363636
std     1.537412          9.913260        89.442719   3267.192289
min     1.000000          9.000000       100.000000   1000.000000
25%     2.000000         17.500000       100.000000   2600.000000
50%     3.000000        

In [3]:
# 5. Análisis de Inventarios #

# Combinar dataframes de ventas e inventario por tiendas y productos
# "inner" se usa para mantener solo las combinaciones que existen en ambos DataFrames
ventas_e_inventario = pd.merge(ventas,
                               inventario,
                               on=['ID_Tienda', 'Producto'],
                               how='inner',
                               suffixes=('_Venta', '_Inventario'))

# Calcular rotación de inventarios (ventas totales / stock disponible) por tienda
ventas_e_inventario["Rotación"] = ventas_e_inventario["Cantidad_Vendida"] / ventas_e_inventario["Stock_Disponible"]
print('\nRotación de inventarios totales por tienda:')
print(ventas_e_inventario.groupby(['ID_Tienda'])['Rotación'].sum())

# Detectar niveles de inventario críticos (ventas menor al 10% del stock)
UMBRAL_INVENTARIO_CRITICO = 0.1
print('\nTiendas con niveles críticos de inventario:')
print(ventas_e_inventario[ventas_e_inventario['Rotación'] < UMBRAL_INVENTARIO_CRITICO])


Rotación de inventarios totales por tienda:
ID_Tienda
1    0.775000
2    1.055556
3    0.833333
4    1.000000
5    1.090000
Name: Rotación, dtype: float64

Tiendas con niveles críticos de inventario:
    ID_Tienda    Producto  Cantidad_Vendida  Precio_Unitario Fecha_Venta  \
10          5  Producto D                 9              300  2023-01-14   

    Ingresos  Stock_Disponible Fecha_Actualización  Rotación  
10      2700               100          2023-01-14      0.09  


In [4]:
# 6. Satisfacción del cliente #

# Enumerar tiendas con baja satisfacción (< 60%)
UMBRAL_SATISFACCION = 60
print('Tiendas con nivel de satisfacción bajo (se recomienda mejorar su rendimiento):')
print(satisfaccion[satisfaccion['Satisfacción_Promedio'] < UMBRAL_SATISFACCION])

Tiendas con nivel de satisfacción bajo (se recomienda mejorar su rendimiento):
   ID_Tienda  Satisfacción_Promedio Fecha_Evaluación
4          5                     55       2023-01-15


In [5]:
# 7. Operaciones con Numpy #

# Calcular mediana y desviación estándar de las ventas totales
ventas_totales = ventas['Cantidad_Vendida'].to_numpy()
mediana_vt = np.median(ventas_totales)
std_vt = np.std(ventas_totales)
print(f'Mediana de las ventas totales: {mediana_vt}')
print(f'Desviación estándar de las ventas totales: {std_vt:.5f}')

# Calcular mediana y desviación estándar de los ingresos totales
ingresos_totales = ventas['Ingresos'].to_numpy()
mediana_it = np.median(ingresos_totales)
std_it = np.std(ingresos_totales)
print(f'\nMediana de los ingresos totales: {mediana_it}')
print(f'Desviación estándar de los ingresos totales: {std_it:.5f}')

# Generar proyecciones de futuras ventas
MAX_VENTAS = 100
np.random.seed(42)  # Configurar semilla por reproducibilidad
futuras_ventas = np.random.randint(MAX_VENTAS, size=(len(ventas_totales)))
print(f'\nProyecciones de futuras ventas: {futuras_ventas}')

Mediana de las ventas totales: 25.0
Desviación estándar de las ventas totales: 9.45192

Mediana de los ingresos totales: 3000.0
Desviación estándar de los ingresos totales: 3115.14562

Proyecciones de futuras ventas: [51 92 14 71 60 20 82 86 74 74 87]
